In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.set_option("display.max_rows", 999)

In [3]:
trade = pd.read_csv("../data/trade-information.csv")
ir = pd.read_csv("../data/trade-price-ir-vegas.csv")
swap = pd.read_csv("../data/market-data-swap-rates.csv")
swaption = pd.read_csv("../data/market-data-swaption-vols.csv")

In [4]:
#compare = ir.loc[ir["Trade Name"].isin(["dummyTrade1", "dummyTrade2"])]

In [5]:
#compare.head(999)

In [6]:
trade['bound width'] = trade['upper_bound'] - trade['lower_bound'] 

trade['month_pay_frequency'] = trade['pay_frequency'].str.removesuffix("M") # remove M from string
trade['month_pay_frequency'] = trade['month_pay_frequency'].astype(int) # convert to int


trade['year_maturity'] = trade['maturity'].str.removesuffix("Y") # remove Y from string
trade['year_maturity'] = trade['year_maturity'].astype(int)  # convert to int


In [7]:
trade

,trade name,underlying,pay_frequency,maturity,lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity
0,dummyTrade1,USD: CMS:2Y,6M,5Y,0.0042,0.0379,0.0337,6,5
1,dummyTrade2,USD: CMS:2Y,4M,5Y,0.0042,0.0379,0.0337,4,5
2,dummyTrade3,USD: CMS:5Y,6M,5Y,0.0044,0.0397,0.0353,6,5
3,dummyTrade4,USD: CMS:5Y,4M,5Y,0.0044,0.0397,0.0353,4,5
4,dummyTrade5,USD: CMS:10Y,4M,5Y,0.0046,0.0417,0.0371,4,5
5,dummyTrade6,USD: CMS:10Y,3M,5Y,0.0046,0.0417,0.0371,3,5
6,dummyTrade7,USD: CMS:2Y,6M,10Y,0.0042,0.0379,0.0337,6,10
7,dummyTrade8,USD: CMS:2Y,4M,10Y,0.0042,0.0379,0.0337,4,10
8,dummyTrade9,USD: CMS:5Y,6M,10Y,0.0044,0.0397,0.0353,6,10
9,dummyTrade10,USD: CMS:5Y,4M,10Y,0.0044,0.0397,0.0353,4,10


In [8]:
swap

,Date,Start Date,Tenor,Swap Rate
0,13/1/2021,13/1/2022,10y,1.081333
1,13/1/2021,13/1/2022,15y,1.265960
2,13/1/2021,13/1/2022,18M,0.221625
3,13/1/2021,13/1/2022,1y,0.195474
4,13/1/2021,13/1/2022,2y,0.279370
...,...,...,...,...
63175,12/1/2024,12/1/2039,2y,3.424505
63176,12/1/2024,12/1/2039,3y,3.393758
63177,12/1/2024,12/1/2039,4y,3.358897
63178,12/1/2024,12/1/2039,5y,3.319531


In [9]:
ir
ir.columns

Index(['Value Date', 'Trade Name', 'Trade Currency', 'Zero Rate Shock', 'TV',
       'Expiry Bucket', 'Expiry Date', 'Tenor Bucket', 'Vega'],
      dtype='object')

In [10]:
# dictionary storing for tenor
dummy2YCMS = {'dummyTrade1': 2, 'dummyTrade2': 2, 'dummyTrade7': 2, 'dummyTrade8': 2}
dummy5YCMS = {'dummyTrade3': 5, 'dummyTrade4': 5, 'dummyTrade9': 5, 'dummyTrade10': 5}
dummy10YCMS = {'dummyTrade5': 10, 'dummyTrade6': 10, 'dummyTrade11': 10, 'dummyTrade12': 10}
YtoInt = {2:"2y", 5: "5y", 10: "10y"}

dummyTenorCMS = {**dummy2YCMS, **dummy5YCMS, **dummy10YCMS}

In [50]:
startfilter = swap['Start Date'].isin(ir['Value Date']) 
timefilter = swap['Tenor'].isin(YtoInt.values())
swap_rate = swap[startfilter & timefilter]

swap_rate = swap_rate.drop_duplicates(['Start Date', 'Tenor'], keep = 'last')
swap_rate


,Date,Start Date,Tenor,Swap Rate
2682,2/3/2021,2/9/2022,10y,1.554418
2686,2/3/2021,2/9/2022,2y,0.600688
2689,2/3/2021,2/9/2022,5y,1.135497
4707,7/4/2021,7/10/2022,10y,1.743677
4711,7/4/2021,7/10/2022,2y,0.762730
4714,7/4/2021,7/10/2022,5y,1.359913
13527,6/9/2021,6/9/2022,10y,1.296478
13531,6/9/2021,6/9/2022,2y,0.599896
13534,6/9/2021,6/9/2022,5y,1.002287
13608,7/9/2021,7/9/2022,10y,1.342631


In [56]:
ir['CMS Tenor'] = ir['Trade Name'].map(dummyTenorCMS).map(YtoInt)
zero = ir.loc[ir['Zero Rate Shock'] == 0]
# map the swap rate to the current zero rate shock 0 swap rate, fill zero rate change later


zero = zero.merge(swap_rate, left_on = ['Value Date', 'CMS Tenor'], right_on = ['Start Date', 'Tenor'], how = "left")


zero.drop(columns = ['Tenor', 'Date', 'Start Date'], inplace = True)
zero
zero.head(1000)
zero.columns

Index(['Value Date', 'Trade Name', 'Trade Currency', 'Zero Rate Shock', 'TV',
       'Expiry Bucket', 'Expiry Date', 'Tenor Bucket', 'Vega', 'CMS Tenor',
       'Swap Rate'],
      dtype='object')

In [57]:
ir = ir.merge(zero, on = ['Value Date', 'Trade Name', 'Zero Rate Shock', 'Trade Currency',  'TV',
       'Expiry Bucket', 'Expiry Date', 'Tenor Bucket', 'Vega', 'CMS Tenor'],  how = "left")

In [58]:
ir

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate
0,2/9/2022,dummyTrade1,USD,-100,-227907.0988,1y,4/9/2023,10y,1.962246,2y,NaN
1,2/9/2022,dummyTrade1,USD,-50,-222208.4010,1y,4/9/2023,10y,-3.812341,2y,NaN
2,2/9/2022,dummyTrade1,USD,-25,-218960.9280,1y,4/9/2023,10y,4.471006,2y,NaN
3,2/9/2022,dummyTrade1,USD,-10,-216872.4301,1y,4/9/2023,10y,4.333398,2y,NaN
4,2/9/2022,dummyTrade1,USD,-5,-216146.3103,1y,4/9/2023,10y,5.679687,2y,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1048570,2/2/2023,dummyTrade6,USD,75,-197572.2089,15y,2/2/2038,10y,0.104718,10y,NaN
1048571,2/2/2023,dummyTrade6,USD,100,-186933.5679,15y,2/2/2038,10y,0.238781,10y,NaN
1048572,2/2/2023,dummyTrade7,USD,-100,-680981.2765,15y,2/2/2038,10y,2.552405,2y,NaN
1048573,2/2/2023,dummyTrade7,USD,-50,-656248.2494,15y,2/2/2038,10y,2.687706,2y,NaN


In [60]:
ir.to_csv("new ir.csv")
# have bug: duplicated data with different swap rates